In [105]:
import pandas as pd
import torch
import matplotlib as mpl
import matplotlib.pyplot as plt
from datasets import DatasetDict, load_from_disk, Dataset, concatenate_datasets
import re

In [106]:
import json
with open('/opt/ml/input/data/KorQuAD_2.1/train/korquad2.1_train_00.json','r') as f:
    train_data = json.load(f)

In [107]:
train_data.keys()

dict_keys(['version', 'data'])

In [108]:
type(train_data)

dict

In [109]:
train_data['version']

'KorQuAD_2.0_train'

In [110]:
len(train_data['data'])

1000

In [111]:
train_data['data'][0].keys()

dict_keys(['title', 'url', 'context', 'raw_html', 'qas'])

In [112]:
train_data['data'][999].keys()

dict_keys(['title', 'url', 'context', 'raw_html', 'qas'])

In [392]:
# http://zeany.net/46
def preprocess_context(example):
    body = re.sub('<title.*?>.*?</title>', '', example, 0, re.I|re.S)
    body = re.sub('<table.*?>.*?</table>', '', body, 0, re.I|re.S)
    body = re.sub('<td.*?>.*?</td>', '', body, 0, re.I|re.S)
    body = re.sub('<li.*?>.*?</li>', '', body, 0, re.I|re.S)
    body = re.sub('<ol.*?>.*?</ol>', '', body, 0, re.I|re.S)
    text = re.sub('<.+?>', '', body, 0, re.I|re.S)
    text = text.replace("[편집]","")
    text = text.split("같이 보기")[0]
    text = text.split("참고 자료")[0]
    text = text.split("외부 링크")[0]
    text = text.split("원본 주소")[0]
    text = text.split("각주")[0]
    text = text.split("참고 문헌")[0]
    text = text.split("함께 보기")[0]
    text = text.split("관련 서적")[0]
    text = text.split("관련 항목")[0]
    space = re.sub('&nbsp;| |\t|\r|\n|\xa0', ' ', text)
    one_space = re.sub(' +', ' ', space)
    return one_space

In [393]:
import json
whole_list = []
train_dict = {}
answer_dict = {}
c = 0
for j in range(0,39):
    file_j = ('0'+str(j))[-2:]
    with open('/opt/ml/input/data/KorQuAD_2.1/train/korquad2.1_train_{}.json'.format(file_j),'r') as f:
        train_data = json.load(f)
        for i in range(len(train_data['data'])):
            answers = train_data['data'][i]['qas'][0]
            answer_dict['answer_start'] = [len(preprocess_context(train_data['data'][i]['context'][:answers['answer']['answer_start']]).lstrip())]
            answer_dict['text'] = [preprocess_context(answers['answer']['text']).strip()]
            count_id = ('000000' + str(c))[-6:]
            train_dict['title']=train_data['data'][i]['title']
            train_dict['context']= preprocess_context(train_data['data'][i]['context']).strip()
            train_dict['question']=answers['question']
            train_dict['id']='korquad-{}-{}'.format(file_j,count_id)
            train_dict['answers']=answer_dict
            train_dict['document_id']=int(answers['id'])
            train_dict['__index_level_0__']=0
            if len(answer_dict['text'][0])>0:
                if (len(train_dict['context']) <= 2000)&(len(answer_dict['text']) <= 80):
                    if len(train_dict['context']) > answer_dict['answer_start'][0]:
                        if train_dict['context'][answer_dict['answer_start'][0]] != answer_dict['text'][0][0]:
                            continue
                        else:
                            whole_list.append(train_dict)
                    else:
                        continue
                else:
                    continue
            else:
                continue
            answer_dict={}
            train_dict={}
            c = c + 1

In [394]:
len(whole_list)

12876

In [398]:
for idx,i in enumerate(range(0,14000,2000)):
    with open('/opt/ml/input/data/KorQuAD_2.1/train/preprocessed/train_under2000_{}.json'.format(idx),'w') as outfile:
        json.dump(whole_list[i:i+2000],outfile)
        print(i,i+2000)

0 2000
2000 4000
4000 6000
6000 8000
8000 10000
10000 12000
12000 14000


In [397]:
import random
whole_list[random.randint(1,len(whole_list))]

{'title': '이봉조',
 'context': '생애 경상남도 남해에서 출생하였고 지난날 한때 경상남도 진주에서 잠시 유아기를 보낸 적이 있는 그는 일찍이 미국의 재즈 음악에 심취하였다. 진주고등보통학교를 졸업한 후 1952년 한양공대 건축공학과에 입학했다. 대학 시절이던 1954년부터 미8군에서 취미 활동으로 재즈 색소폰을 연주 및 공연하였으며 2년간 휴학한 뒤 1958년 대학을 나왔다. 서울특별시청 토목과 공무원으로 재직하면서 틈틈이 미8군 무대에서 재즈 테너 색소폰을 연주하며 테너 색소포니스트 엄토미(嚴吐美)의 문하생으로서 단연 두각을 나타내었다. 1961년 서울특별시청 공무원직에서 퇴직하였고 그 후 주직업적으로 재즈 음악가의 길을 걸었다. 작곡가의 입장으로써는 현미(玄美), 정훈희(鄭薰姬) 등의 가수를 발굴하였고 뮤지컬 배우 출신의 윤복희(尹福姬)를 훈련시켜 가수로 재발탁하였다. 1963년에는 영화 《가정교사》로 영화 음악감독 데뷔하였다. 1965년 영화 《광야의 호랑이》로 영화 음악연출가 데뷔하였으며 1967년 영화 《안개》로 영화음악연주감독 데뷔했다. 그 후 1970년대에는 여러 국제가요제에서도 작곡가로써 자신의 명성을 떨쳤으며 개신교 신자라는 입장으로 색소폰 연주곡 CCM 가스펠 음반도 다수 발표하였다. 그는 부인 노전숙과 연애 끝에 1956년 결혼하였고 3남 3녀를 낳았다. 순탄할줄 알았던 결혼 생활은 1958년에 처음 시작된 자신의 외도로 인하여 나중엔 이봉조와 노전숙이 따로 홀로 살게되었다고 한다. 이후 이봉조는 1987년 8월 31일 심장마비로 사망하였다. 학력 별칭 재즈 테너 색소폰 연주자로서 두각을 나타내면서 한국의 스탠 게츠라는 별칭을 얻었다.',
 'question': '이봉조의 스승이었던 테너 색소포니스트는 누구였을까?',
 'id': 'korquad-29-009949',
 'answers': {'answer_start': [255], 'text': ['엄토미']},
 'document_id': 14283,
 '__index_

In [403]:
import json
dev_list = []
train_dict = {}
answer_dict = {}
c = 0
for j in range(0,5):
    file_j = ('0'+str(j))[-2:]
    with open('/opt/ml/input/data/KorQuAD_2.1/dev/korquad2.1_dev_{}.json'.format(file_j),'r') as f:
        train_data = json.load(f)
        for i in range(len(train_data['data'])):
            answers = train_data['data'][i]['qas'][0]
            answer_dict['answer_start'] = [len(preprocess_context(train_data['data'][i]['context'][:answers['answer']['answer_start']]).lstrip())]
            answer_dict['text'] = [preprocess_context(answers['answer']['text']).strip()]
            count_id = ('000000' + str(c))[-6:]
            train_dict['title']=train_data['data'][i]['title']
            train_dict['context']= preprocess_context(train_data['data'][i]['context']).strip()
            train_dict['question']=answers['question']
            train_dict['id']='korquad-{}-{}'.format(file_j,count_id)
            train_dict['answers']=answer_dict
            train_dict['document_id']=int(answers['id'])
            train_dict['__index_level_0__']=0
            if len(answer_dict['text'][0])>0:
                if (len(train_dict['context']) <= 2000)&(len(answer_dict['text']) <= 80):
                    if len(train_dict['context']) > answer_dict['answer_start'][0]:
                        if train_dict['context'][answer_dict['answer_start'][0]] != answer_dict['text'][0][0]:
                            continue
                        else:
                            dev_list.append(train_dict)
                    else:
                        continue
                else:
                    continue
            else:
                continue
            answer_dict={}
            train_dict={}
            c = c + 1

In [404]:
len(dev_list)

1513

In [408]:
dev_list[0]

{'title': '공법',
 'context': '구별실익 강력한 중앙집권적 전제국가가 존재하던 로마시대, 근대에는 공사법 구별이 엄격하였다. 반면에 개인주의가 발달한 시대에는 사법의 영역이 확장되었다. 최근 복지국가사상이 발달하면서 공사법 구별이 엄격해지고 있다. 즉, 사적자치가 발달하면 공사법이 부정되고, 국가의 권력이 강력해지면 공사법의 구별이 엄격해진다. 이를 통해서, 공법을 사법에서 굳이 분리하려는 이유는, 국가와 개인이 소송할 때는, 개인과 개인이 소송할 때보다 국가에게 더 유리하게 판결을 하기 위해 공법이 사법에서 분리된다는 것을 알 수 있다. 영미법은 1천년이 넘도록 공사법 구별이 없이 사법으로만 규율하였는데, 20세기에 들어서 공법이 등장하고 있다는 것은, 영미법에서 개인주의나 사적자치가 후퇴하고 국가주의가 강화되고 있다는 의미로도 해석될 수 있다. 공법과 사법의 구별은 구체적 법률관계에 적용할 법규나 법원칙을 결정하기 위해, 또한 분쟁해결을 위한 쟁송수단의 선택과 결정을 위해서 필요하다. 공법과 사법의 관계 최근에 새로이 정립되는 사회법(노동법 및 경제관계법)은 사법과 공법이 융합된 것으로 사법의 공법화 경향을 보여준다. 공법·사법의 이원적 체계를 인정하지 않고 보통법의 지배 원리에 의한 일원적 체계의 영미법계에서는 20세기에 들어오면서 행정법의 성립과 함께 공법과 사법의 구별 문제가 대두되기 시작하였다. 구별 부정설 구별 부정론은 공법 관계도 그 본질에서는 사법 관계와 같은 법률관계인 점에서 차이가 없으며, 노동법·사회법·경제법 등과 같이 공법과 사법의 어느 하나에만 속하지 않는 새로운 법 현상이 출현하는 것을 이유로 공법과 사법의 구별을 부정하는 견해이다. 구별 긍정설 구별 긍정론 가운데 실체법상의 구별 필요성은 우리의 현행 법질서는 공법 관계인가 사법 관계인가에 따라 적용될 법규나 법원칙을 달리하므로 구체적 법률관계에 적용할 법규나 법원칙을 결정하기 위하여 공법과 사법의 구별은 필요하다. 예컨대 행정 주체와 사인(私人)간에 계약을 체결

In [407]:
for idx,i in enumerate(range(0,1530,510)):
    with open('/opt/ml/input/data/KorQuAD_2.1/dev/preprocessed/valid_under2000_{}.json'.format(idx),'w') as outfile:
        json.dump(dev_list[i:i+510],outfile)
        print(i,i+510)

0 510
510 1020
1020 1530


In [47]:
ds = Dataset.from_json('/opt/ml/input/data/KorQuAD_2.1/train/train.json')

Using custom data configuration default-5f30fd53ca4be2a5
Found cached dataset json (/opt/ml/.cache/huggingface/datasets/json/default-5f30fd53ca4be2a5/0.0.0)


In [48]:
dv = Dataset.from_json('/opt/ml/input/data/KorQuAD_2.1/dev/valid.json')

Using custom data configuration default-56fe0ff52f29f51d
Found cached dataset json (/opt/ml/.cache/huggingface/datasets/json/default-56fe0ff52f29f51d/0.0.0)


In [98]:
ds['context'][0][ds['answers'][0]['answer_start'][0]]

'나'

In [99]:
ds['answers'][0]['text'][0][0]

'나'

In [100]:
train_dataset = load_from_disk('/opt/ml/input/data/train_dataset')

In [101]:
train_dataset

DatasetDict({
    train: Dataset({
        features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'],
        num_rows: 3952
    })
    validation: Dataset({
        features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'],
        num_rows: 240
    })
})

In [102]:
concatenate_datasets([train_dataset['train'],ds])

Dataset({
    features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'],
    num_rows: 42448
})

In [103]:
DatasetDict({'train' : ds, 'validation' : dv})

DatasetDict({
    train: Dataset({
        features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'],
        num_rows: 38496
    })
    validation: Dataset({
        features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'],
        num_rows: 4736
    })
})